# Getting started with grain

grain is dynamic autograd library on CPU/CUDA devices for D language. While grain's autograd mechanism is dynamic, D compilers can strongly optimize  and validate your scripts statically. This combination gives you great experience. At beginning, you might miss P language's popular REPL (interactive environment). D also offers the similar one here.

## how to run grain in jupyter

This installation guide is tested with

- dmd 2.081.2 (recommend [the official installer](https://dlang.org/install.html))
- jupyter 4.4.0 (recommend anaconda)

and install d kernel and engine as follows
``` bash
git clone https://github.com/ShigekiKarita/grain
cd grain
jupyter kernelspec install ./example/third/jupyterd --user
dub build --config=jupyterd --compiler=dmd
export PATH=`pwd`:$PATH
jupyter notebook
```

Then you can create a new notebook with 'd' kernel or open existing 'd' notebooks.

## expected readers

Generally this tutorial is for D users. You can learn D easility in https://tour.dlang.org/tour/ Optionally, experiences of numerical library in D (mir) or in python (numpy, pytorch) help you understanding grain.

## reference

- Main topics in this tutorial are derived from https://pytorch.org/tutorials

## TODO

- DUB extension for drepl like `%load_dub grain`

# mir.ndslice.Slice and numir

Before introducing grain, we will first implement the network using mir. Mir provides an n-dimensional slice value, and many functions for manipulating these slices. Mir is a generic framework for scientific computing; it does not know anything about computation graphs, or deep learning, or gradients. However we can easily use mir to fit a two-layer network to random data by manually implementing the forward and backward passes through the network using mir operations:

In [1]:
// TODO hand-written 2 layer NN implementation like example/char_rnn_hand.d
import mir.ndslice;

mir

# grain.autograd.Variable

In the above examples, we had to manually implement both the forward and backward passes of our neural network. Manually implementing the backward pass is not a big deal for a small two-layer network, but can quickly get very hairy for large complex networks.

Thankfully, we can use automatic differentiation to automate the computation of backward passes in neural networks. The autograd package in grain provides exactly this functionality. When using autograd, the forward pass of your network will define a computational graph; nodes in the graph will be `Variable`, and edges will be functions that produce output Variable from input Variable. In addition to Slice feature, Variable contains autograd history (simply previously forwarded `Function` object that also implements its backward function) as `grain.autograd.Backprop` type. Backpropagating through this graph then allows you to easily compute gradients. Moreover, unlike mir.ndslice only supports CPU memory slices, grain supports both CPU and CUDA memory slices.

This sounds complicated, it’s pretty simple to use in practice. Each Slice represents a node in a computational graph. If x is a Slice that has x.requiresGrad=True then x.grad is another memory holding the gradient of x with respect to some scalar value (and x.gradSlice hold a slice of the gradient).

Here we use `grain.autograd.Variable` and autograd to implement our two-layer network; now we no longer need to manually implement the backward pass through the network:

In [1]:
// TODO fix empty line outputs
import std.stdio;
import grain;
import mir.random;
import mir.ndslice;
{
    // enable autograd mode
    backprop = true;
    alias ftype = float;
    alias device = HostStorage; // DeviceStorage

    auto learningRate = 1e-6;
    auto batchSize = 6;
    auto inputDim = 10;
    auto hiddenDim = 10;
    auto outputDim = 5;
    
    auto genRand(bool requiresGrad = false, size_t dim)(size_t[dim] s...) {
        return iota(s)
            .map!(_ => rand!ftype)
            .slice
            .variable(requiresGrad)
            .to!device;
    }

    // create random tensors to hold input and outputs
    auto x = genRand(batchSize, inputDim);
    auto y = genRand(batchSize, outputDim);

    auto w1 = genRand!true(inputDim, hiddenDim);
    auto w2 = genRand!true(hiddenDim, outputDim);

    // compose NN sequence like torch with UFCS
    auto y_pred = x
        .matMul(w1)
        .relu
        .matMul(w2);

    auto loss = (y - y_pred).pow(2).sum;
    loss.backward();
    // see backprop start to end
    writeln(loss);
    writeln(w1.gradSlice);
}


std
grain
mir
mir






































Variable!(float, dim=0, HostStorage(T))(data=[63.48], shape=[], strides=[])
[[14.2033, -1.17908, 2.58509, 9.15378, -1.33056, 11.0214, 13.6442, 0.34182, -6.33891, 5.69314], [7.90239, -3.00276, -5.48996, -7.46606, -1.01709, -1.63814, 0.854643, 2.12084, -2.06309, -3.13116], [7.51052, -2.48702, -1.88054, -0.980722, 4.25312, 6.66366, 0.285049, 4.7007, 0.390635, -6.10982], [-16.213, 2.70848, -1.50313, -8.83716, -4.71922, -17.0967, -11.6338, -5.59637, 2.8569, 0.653741], [9.30754, 0.524146, 6.76562, 15.4717, -0.532595, 14.4874, 12.3955, -0.921544, -5.03076, 7.48531], [-16.5343, 1.46255, 0.0558762, -7.58883, 1.91899, -10.8742, -7.75577, 0.716138, 8.34946, 1.01745], [7.73426, -1.01884, -0.609923, 1.7603, -2.78924, 2.95332, 4.13461, -1.15211, -4.64064, 1.37203], [-3.71636, 0.0632498, 6.16413, 2.98068, -1.60251, 1.78221, 0.894082, -1.51496, 0.40581, 6.59821], [-6.50851, 2.00647, 5.34826, 1.07636, -3.85597, -4.88909, -5.03178, -5.56445, -1.63

In [1]:
// numpy-like broadcasting is available
static assert(!__traits(compiles, [[0.1f], [0.3f]].variable + [0.2f].variable));
// but you need to make them same dimensions
[[0.1f, 0.2f], [0.3f, 0.4f]].variable + [[0.2f]].variable




Variable!(float, dim=2, HostStorage(T))(data=[0.3, 0.4, 0.5, 0.6], shape=[2, 2], strides=[2, 1])

## Defining new autograd functions

Under the hood, each primitive autograd operator is really two functions that operate on Variables. The forward function computes output Variables from input Variable. The backward function receives the gradient of the output Variables with respect to some scalar value, and computes the gradient of the input Variables with respect to that same scalar value.

In grain, we can easily define our own autograd operator by defining a `Function` with `grain.functions.common.FunctionCommon` mixin and implementing the forward and backward functions. We can then use our new autograd operator by constructing an instance and calling it like a function, passing Variables containing input data.

- you can find many examples in `grain.functions` module

In this example we define our own custom autograd function for performing the ReLU nonlinearity, and use it to implement our two-layer network:

In [1]:
/// rectified linear unit nonlinearity
struct MyReLU(T, size_t dim) {

    mixin FunctionCommon;
    Variable!(T, dim, HostStorage) hx;

    auto forward(Variable!(T, dim, HostStorage) x) {
        import mir.ndslice : each;
        this.hx = x.dup;
        auto y = x.dup;
        y.sliced.each!((ref a) {
            if (a < 0)
                a = 0;
        });
        return y;
    }

    auto backward(Variable!(T, dim, HostStorage) gy) {
        auto gx = gy.dup;
        foreach (i; 0 .. gx.data.length) {
            if (this.hx.data[i] < 0.0)
                gx.data[i] = 0.0;
        }
        return gx;
    }
}


























MyReLU